<a href="https://colab.research.google.com/github/tlancaster6/AquaMVS/blob/main/docs/tutorial/benchmark.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Benchmarking Reconstruction Pathways

This tutorial demonstrates how to benchmark AquaMVS reconstruction pathways,
compare LightGlue and RoMa matching strategies, and visualize performance
and quality metrics using the Python API.

AquaMVS supports three reconstruction pathways:
- **LG+SP sparse**: Fast sparse reconstruction using SuperPoint features
- **LG+SP full**: Dense stereo seeded by LightGlue sparse matches
- **RoMa full**: Dense stereo seeded by RoMa dense matches

> **Why no RoMa sparse?** RoMa produces dense per-pixel warp fields, so
> running it in sparse mode pays the same expensive matching cost but
> discards most of the output to extract keypoints. Use LG+SP sparse for
> efficient sparse reconstruction instead.

## Setup

In [1]:
import importlib.util
import subprocess
import sys

if importlib.util.find_spec("aquamvs") is None:
    subprocess.run(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "torch",
            "torchvision",
            "--index-url",
            "https://download.pytorch.org/whl/cpu",
            "-q",
        ],
        check=True,
    )
    subprocess.run(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "git+https://github.com/cvg/LightGlue.git@edb2b83",
            "git+https://github.com/tlancaster6/RoMaV2.git",
            "aquamvs",
            "-q",
        ],
        check=True,
    )

In [2]:
import os
import urllib.request
import zipfile
from pathlib import Path

DATASET_URL = "https://zenodo.org/records/18702024/files/aquamvs-example-dataset.zip"
DATASET_DIR = Path("aquamvs-example-dataset")

if not DATASET_DIR.exists():
    print("Downloading example dataset...")
    urllib.request.urlretrieve(DATASET_URL, "aquamvs-example-dataset.zip")
    with zipfile.ZipFile("aquamvs-example-dataset.zip") as zf:
        zf.extractall(DATASET_DIR)
    os.remove("aquamvs-example-dataset.zip")
    print("Done.")
else:
    print(f"Dataset already present at {DATASET_DIR}")

# Change into the dataset directory so relative config paths resolve correctly
os.chdir(DATASET_DIR)
CONFIG_PATH = Path("config.yaml")

Dataset already present at aquamvs-example-dataset


## Running the Benchmark

The `run_benchmark` function runs all three reconstruction pathways on a
single frame and returns a `BenchmarkResult` with per-pathway timing,
point count, and density metrics.

| Pathway | Matcher | Mode | Description |
|---------|---------|------|-------------|
| RoMa full | RoMa | full | Dense stereo seeded by RoMa dense matches |
| LG+SP full | LightGlue/SuperPoint | full | Dense stereo seeded by sparse matches |
| LG+SP sparse | LightGlue/SuperPoint | sparse | Triangulation from sparse features only |

> **Tip:** You can also run benchmarks from the command line:
> `aquamvs benchmark config.yaml --frame 0`

In [ ]:
from aquamvs.benchmark.runner import run_benchmark

benchmark_result = run_benchmark(config_path=CONFIG_PATH, frame=0)

print(
    f"Benchmarked {len(benchmark_result.results)} pathways on frame {benchmark_result.frame}"
)
print()

for pw in benchmark_result.results:
    total_s = pw.timing.total_time_ms / 1000.0
    print(
        f"{pw.pathway_name:20s}: {total_s:.1f}s, "
        f"{pw.point_count:,} points, "
        f"density={pw.cloud_density:.0f} pts/m\u00b2"
    )

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\tucke\PycharmProjects\AquaMVS\src\aquamvs\projection\refractive.py:57: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:39.)
  self.K = self.K.to(device)
Using cache found in C:\Users\tucke/.cache\torch\hub\facebookresearch_dinov3_adc254450203739c8149213a7a69d8d905b4fcfa


2026-02-21 10:04:56 INFO     romav2.romav2 - romav2:116 in __init__ - RoMa v2 initialized.

## Visualizing Results

The plots below compare the four pathways on two metrics:

- **Total runtime** (left): How long each pathway takes end-to-end
- **Point count** (right): How many 3D points each pathway produces after fusion

Sparse pathways are typically faster but produce fewer points. Full (dense) pathways
take longer but produce denser reconstructions.

In [ ]:
import matplotlib.pyplot as plt

names = [pw.pathway_name for pw in benchmark_result.results]
total_times_s = [pw.timing.total_time_ms / 1000.0 for pw in benchmark_result.results]
point_counts = [pw.point_count for pw in benchmark_result.results]

# Professional color palette: coral for RoMa, teal for LG+SP
colors = ["#e07b54", "#2e86ab", "#1a5276"]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Total runtime
axes[0].bar(names, total_times_s, color=colors)
axes[0].set_title("Total Runtime per Pathway", fontsize=13)
axes[0].set_ylabel("Time (s)")
axes[0].set_xlabel("Pathway")
axes[0].tick_params(axis="x", rotation=15)
for i, v in enumerate(total_times_s):
    axes[0].text(i, v + 0.1, f"{v:.1f}s", ha="center", fontsize=9)

# Point count
axes[1].bar(names, point_counts, color=colors)
axes[1].set_title("Reconstructed Point Count per Pathway", fontsize=13)
axes[1].set_ylabel("Points")
axes[1].set_xlabel("Pathway")
axes[1].tick_params(axis="x", rotation=15)
for i, v in enumerate(point_counts):
    axes[1].text(i, v + max(point_counts) * 0.01, f"{v:,}", ha="center", fontsize=9)

plt.tight_layout()
plt.show()

## Stage Timing Breakdown

The stacked bar chart below shows how each pathway spends its time across pipeline stages.
This is useful for identifying bottlenecks and understanding where the pathways differ.

Key stages:
- **undistortion**: Lens distortion removal (same cost across all pathways)
- **sparse_matching** / **dense_matching**: Feature extraction and matching
- **depth_estimation**: Plane-sweep stereo (only in `full` mode)
- **fusion**: Multi-view depth map merging (only in `full` mode)
- **surface**: Surface reconstruction from the point cloud

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ordered stages and display labels
STAGE_ORDER = [
    ("undistortion", "Undistortion"),
    ("sparse_matching", "Sparse Matching"),
    ("dense_matching", "Dense Matching"),
    ("depth_estimation", "Depth Estimation"),
    ("fusion", "Fusion"),
    ("surface", "Surface"),
]
STAGE_COLORS = ["#4e8098", "#90c2e7", "#6baed6", "#e07b54", "#f4a261", "#a8dadc"]

pathway_names = [pw.pathway_name for pw in benchmark_result.results]
x = np.arange(len(pathway_names))

fig, ax = plt.subplots(figsize=(12, 6))
bottoms = np.zeros(len(pathway_names))

for (stage_key, stage_label), color in zip(STAGE_ORDER, STAGE_COLORS, strict=False):
    stage_times = []
    for pw in benchmark_result.results:
        stage = pw.timing.stages.get(stage_key)
        stage_times.append(stage.wall_time_ms / 1000.0 if stage is not None else 0.0)
    ax.bar(x, stage_times, bottom=bottoms, color=color, label=stage_label)
    bottoms += np.array(stage_times)

ax.set_title("Stage Timing Breakdown per Pathway", fontsize=13)
ax.set_ylabel("Time (s)")
ax.set_xlabel("Pathway")
ax.set_xticks(x)
ax.set_xticklabels(pathway_names, rotation=15)
ax.legend(loc="upper left", fontsize=9)
plt.tight_layout()
plt.show()

## Comparing Depth Maps

Visual comparison of depth maps between pathways reveals differences in reconstruction
coverage and quality. The benchmark saves outputs for each pathway to a separate
subdirectory under `{output_dir}/benchmark/{pathway_safe_name}/`.

The cell below attempts to load depth maps from two pathways (LG+SP full and RoMa full)
for side-by-side comparison. Note that depth maps are only produced by `full` mode
pathways; sparse mode produces point clouds from triangulation only.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from aquamvs import PipelineConfig

base_config = PipelineConfig.from_yaml(CONFIG_PATH)
base_output = Path(base_config.output_dir)

# Pathway output dirs use safe names (+ -> _, space -> _)
pathway_dirs = {
    "LG+SP full": base_output / "benchmark" / "LG_SP_full",
    "RoMa full": base_output / "benchmark" / "RoMa_full",
}

# Use the first camera from the config
cam = list(base_config.camera_input_map.keys())[0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
loaded_any = False

for ax, (pathway_name, out_dir) in zip(axes, pathway_dirs.items(), strict=False):
    depth_path = out_dir / "frame_000000" / "depth_maps" / f"{cam}.npz"
    if depth_path.exists():
        depth = np.load(depth_path)["depth"]
        im = ax.imshow(depth, cmap="viridis")
        plt.colorbar(im, ax=ax, label="Depth (m)", shrink=0.8)
        ax.set_title(f"{pathway_name} \u2014 {cam}")
        ax.axis("off")
        loaded_any = True
    else:
        ax.set_title(f"{pathway_name}")
        ax.text(
            0.5,
            0.5,
            f"Depth map not found.\nRun the benchmark first or check\n{depth_path}",
            ha="center",
            va="center",
            transform=ax.transAxes,
            fontsize=9,
        )
        ax.axis("off")

if loaded_any:
    plt.suptitle("Depth Map Comparison (full-mode pathways)", fontsize=13)
else:
    plt.suptitle("Run the benchmark above to generate depth maps", fontsize=11)

plt.tight_layout()
plt.show()

## Comparing 3D Reconstructions

If `runtime.viz_enabled: true` is set in your config, each pathway renders
static PNG snapshots of the fused point cloud. The cell below loads these
pre-rendered images for a side-by-side comparison â€” no Open3D needed in
the notebook.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from aquamvs import PipelineConfig

base_config = PipelineConfig.from_yaml(CONFIG_PATH)
base_output = Path(base_config.output_dir)

pathway_dirs = {
    "RoMa full": base_output / "benchmark" / "RoMa_full",
    "LG+SP full": base_output / "benchmark" / "LG_SP_full",
    "LG+SP sparse": base_output / "benchmark" / "LG_SP_sparse",
}

viz_filename = "fused_oblique.png"
images = {}
for name, out_dir in pathway_dirs.items():
    img_path = out_dir / "frame_000000" / "viz" / viz_filename
    if img_path.exists():
        images[name] = plt.imread(str(img_path))

if images:
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=(6 * n, 5))
    if n == 1:
        axes = [axes]
    for ax, (name, img) in zip(axes, images.items(), strict=False):
        ax.imshow(img)
        ax.set_title(name, fontsize=12)
        ax.axis("off")
    plt.suptitle("3D Reconstruction Comparison", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print(
        "No rendered PNGs found. To generate them, set `runtime.viz_enabled: true`\n"
        "in your config.yaml and re-run the benchmark."
    )

## Selecting a Pathway

Use this table to guide your choice of reconstruction pathway:

| Pathway | Speed | Point Density | Use Case |
|---------|-------|---------------|----------|
| LG+SP sparse | Fastest | Low | Quick preview, debugging, sparse structure |
| LG+SP full | Moderate | High | General-purpose dense reconstruction |
| RoMa full | Slowest | Highest | Maximum quality, challenging lighting/texture |

**Recommendations:**

- Start with **LG+SP sparse** to verify your dataset and configuration are correct.
- Use **LG+SP full** for most production reconstructions.
- Switch to **RoMa full** if you see sparse reconstruction failures or low point density
  (often in textureless or highly reflective underwater scenes).

To lock in a pathway for your workflow, set these fields in your `config.yaml`:

```yaml
matcher_type: lightglue  # or roma
pipeline_mode: full       # or sparse
```

Or use the `--preset` flag when initializing: `aquamvs init --preset fast` applies
speed-optimized parameter defaults across all pathways.

## Next Steps

- **[CLI Guide](../cli_guide.md)**: Full reference for `aquamvs` command-line options
- **[End-to-End Tutorial](notebook.ipynb)**: Step-by-step Python API walkthrough
- **[Troubleshooting Guide](../troubleshooting.rst)**: Diagnose common reconstruction issues
- **[API Reference](../api/index.rst)**: Documentation for `aquamvs.benchmark.runner`, `aquamvs.benchmark.report`, and all pipeline modules

To save the benchmark report as a markdown file for later reference:

```python
from aquamvs.benchmark.report import save_markdown_report
from pathlib import Path

report_path = save_markdown_report(benchmark_result, output_dir=Path("./reports"))
print(f"Report saved to {report_path}")
```